In [1]:
library(ChAI)
library(tidyr)
library(data.table)
library(dplyr)
library(parallel)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# Olink

In [2]:
meta_data<-read.csv('/home//jupyter/BRI_Analysis/scRNA/hise_meta_data_2024-01-23_fixed.csv')%>% filter(sample.visitName=='Flu Year 1 Day 0')

In [3]:
sample_kit_id<-meta_data  %>% select(sample.sampleKitGuid) %>% pull()

In [4]:
Olink<-read.csv('/home/jupyter/BRI_Analysis/Olink/Cross-Sectional/Olink.csv')
Olink<-Olink[c('sample.sampleKitGuid','NPX_bridged','Assay')] %>% filter(sample.sampleKitGuid %in% sample_kit_id)

In [5]:
meta_data_olink<-meta_data %>% select (sample.sampleKitGuid,CMV.IgG.Serology.Result.Interpretation,
                                                                                     cohort.cohortGuid,subject.subjectGuid,subject.biologicalSex)

In [6]:
colnames(meta_data_olink)[1]<-'Sample'

In [7]:
Olink_wide<-as.data.frame(Olink%>%  pivot_wider(names_from = sample.sampleKitGuid, values_from = NPX_bridged))
rownames(Olink_wide)<-Olink_wide$Assay

In [8]:
Olink_wide$Assay<-NULL

In [9]:
Olink<-importGeneralModality(Olink_wide, meta_data_olink, sampleColumn = "Sample")


In [10]:
saveRDS(Olink,'Olink.RDS')

# RNA

In [11]:
pbmc_list<-mclapply(meta_data$pbmc_sample_id,function(x){

pbmc <-sceasy::convertFormat(paste0('/home/jupyter/BRI_Analysis/scRNA/h5_cleaned_by_sample/',x,'.h5ad'), from="anndata", to="seurat")

return(pbmc)
},mc.cores=60)

In [12]:
AIFI_L3<-pull(unique(pbmc_list[[1]][[]]['AIFI_L3']))

Loading required package: SeuratObject

Loading required package: sp

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)



In [13]:
length(AIFI_L3)

[1] 71

In [23]:
AIFI_L3

[1] cDC1                                 KLRF1- effector Vd1 gdT             
 [3] ISG+ CD16 monocyte                   CD14+ cDC2                          
 [5] HLA-DRhi cDC2                        GZMK+ Vd2 gdT                       
 [7] CD8aa                                GZMK+ CD56dim NK cell               
 [9] C1Q+ CD16 monocyte                   CD95 memory B cell                  
[11] GZMK- CD27+ EM CD8 T cell            KLRF1+ GZMB+ CD27- EM CD8 T cell    
[13] GZMK- CD56dim NK cell                Memory CD8 Treg                     
[15] Proliferating NK cell                CD4 MAIT                            
[17] Intermediate monocyte                CMP cell                            
[19] SOX4+ Vd1 gdT                        KLRB1+ memory CD8 Treg              
[21] pDC                                  Erythrocyte                         
[23] Core CD14 monocyte                   GZMB+ Vd2 gdT                       
[25] Platelet                             ILC                                 
[27] GZMB- CD27+ EM CD4 T cell            GZMK+ memory CD4 Treg               
[29] Transitional B cell                  CD56bright NK cell                  
[31] Core memory B cell                   KLRB1+ memory CD4 Treg              
[33] CM CD4 T cell                        GZMK+ CD27+ EM CD8 T cell           
[35] Activated memory B cell              IL1B+ CD14 monocyte                 
[37] SOX4+ naive CD4 T cell               BaEoMaP cell                        
[39] KLRF1+ effector Vd1 gdT              CD27- effector B cell               
[41] ISG+ CD56dim NK cell                 ISG+ naive CD8 T cell               
[43] ISG+ CD14 monocyte                   ISG+ naive B cell                   
[45] CD8 MAIT                             CM CD8 T cell                       
[47] Core CD16 monocyte                   Core naive CD8 T cell               
[49] ISG+ naive CD4 T cell                Proliferating T cell                
[51] Plasma cell                          CD27+ effector B cell               
[53] SOX4+ naive CD8 T cell               Core naive CD4 T cell               
[55] ISG+ memory CD8 T cell               Naive Vd1 gdT                       
[57] Core naive B cell                    CLP cell                            
[59] KLRF1- GZMB+ CD27- EM CD8 T cell     Naive CD4 Treg                      
[61] KLRF1- GZMB+ CD27- memory CD4 T cell Early memory B cell                 
[63] GZMB- CD27- EM CD4 T cell            ISG+ memory CD4 T cell              
[65] ISG+ MAIT                            Memory CD4 Treg                     
[67] Adaptive NK cell                     ISG+ cDC2                           
[69] Type 2 polarized memory B cell       DN T cell                           
[71] ASDC                                
71 Levels: ASDC Activated memory B cell Adaptive NK cell ... pDC

In [ ]:
for (celltype in AIFI_L3){
pbmc_subset_list <- mclapply(pbmc_list, function(pbmc) {
    pbmc_subset <- tryCatch({
        subset(pbmc, subset = AIFI_L3 == celltype)
    }, error = function(e) {
        NULL
    })

    if (!is.null(pbmc_subset)) {
        return(pbmc_subset)
    }
}, mc.cores = 60)
pbmc_subset_list <- Filter(function(x) !is.null(x), pbmc_subset_list)
so<-Reduce(
    f = function(x, y) {merge(x, y, merge.data = FALSE)},
    x = pbmc_subset_list 
)
    
so@meta.data<-left_join(so@meta.data,meta_data[c('pbmc_sample_id','sample.sampleKitGuid')])
rownames(so@meta.data)<-so@meta.data$barcodes    
df<-makePseudobulkRNA(
  so,
  'AIFI_L3',
  sampleColumn = "sample.sampleKitGuid",
  cellPopulations = "All",
  numCores =60 ,
  Seurat_format = "SYMBOL",
  TxDb = NULL,
  OrgDb = NULL,
  normalize = FALSE
)
saveRDS(df,paste0("scRNA-object/",pull(so[[]]['AIFI_L3'])[1],'.RDS'))
}

Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
Joining with `by = join_by(pbmc_sample_id)`
